# Red TRANSFORMER  (Desarrollo teórico)

La Red Transformer, descrita en el artículo de 2017 Attention is all you need, fue desarrollada por investigadores de Google, y nació inicialmente como una alternativa al problema de la traducción de texto de un idioma a otro.

En este tipo de red, la totalidad de la secuencia de entrada es procesada en paralelo por la red, a diferencia de las Redes Recurrentes en donde se procesan uno a uno (en serie) los elementos de la secuencia.   

<img src="./img/transformer_1.jpg">

Si observamos la imagen superior, en la que se presenta un diagrama de bloques general de una Red Transformer, se ve como la secuencia de entrada es procesada primero por el codificador (izquierda) y luego es enviada al decodificador (derecha).   

Esta secuencia es inicialmente convertida en una representación numérica usando un embedding

Después se añade una codificación de posición y los vectores resultantes ingresan a la etapa de codificación, que se encarga de extraer la información mas relevante de la secuencia en su idioma original.

La salida de esta etapa se conecta al decodificador, que toma esta información para generar secuencialmente el texto traducido al segundo idioma.

Aunque tiene muchos elementos realmente es sencillo entender cómo funciona.      



A continuación, se procede a analizar en detalle cada módulo de esta red.

## *Embedding* de entrada

Este *"embedding"* es en realidad un algoritmo que convierte el texto en una serie de vectores (*tokens*), es decir en una representación numérica que puede ser “comprendida” por la red.

<img src="./img/embedding-entrada-transformer.jpg">

## Codificador posicional   

Como la secuencia se procesa en paralelo, es necesario indicarle a la red el orden en el que se encuentran las palabras dentro del texto. Esto se logra con el codificador posicional.

Este codificador genera una serie de vectores que se sumarán a los tokens, y que indican la posición relativa de cada token dentro de la secuencia. Para esto se usan funciones senoidales para las posiciones pares, y cosenoidales para las impares, con lo que cada vector generado tendrá un patrón numérico único con la información de la posición.

<img src="./img/codificador-de-posicion-red-transformer.jpeg">

## Codificadores (Bloque de Codificación)
A continuación, nos encontramos con el bloque de codificación, que contiene seis codificadores, todos con una estructura idéntica.     

<img src="./img/bloque-codificacion-red-transformer.jpeg">


### Codificación + Bloque de atención    

Analizamos en detalle uno de estos codificadores.

Cada codificador tiene cuatro elementos:    
- un bloque atencional, 
- un bloque de conexión residual, 
- una red neuronal y 
- otro bloque de conexión residual

<img src="./img/codificador-1-red-transformer.jpeg">    


Observamos en detalle el bloque atencional, que es el más importante de toda la red, responsable de analizar la totalidad de la secuencia de entrada (recordemos que la red la procesa de manera simultánea) y de encontrar relaciones entre varias palabras de esta secuencia.

Por ejemplo, si el texto de entrada es “I love Italian food”, se ve que hay al menos dos posibles asociaciones entre palabras: el verbo “love” y el sujeto (“I”) y el sustantivo “food” asociado al adjetivo “Italian”. Pero además entre estas dos frases (I Love e Italian Food) también hay una asociación.

Lo que hace el bloque atencional es expresar numéricamente las relaciones que existen a diferentes niveles dentro de la secuencia, y luego codifica cada una de ellas con esta información del contexto, indicando así cuáles son los elementos del texto a los que se deben prestar más atención al momento de hacer la traducción.

Esta es precisamente la manera como las redes transformer “comprenden” este contexto para codificar adecuadamente cada palabra.

Para lograr esto en primer lugar los tokens se llevan simultáneamente a tres pequeñas redes neuronales, entrenadas para calcular los vectores `“query”`, `“key”` y `“value”`. Estos vectores son simplemente tres representaciones alternativas de los tokens originales:   


<img src="./img/queries-keys-values-red-transformer.jpeg">   

Después de esto, se coge el `query`de cada token y se compara con cada uno de los `keys` existentes. Esta comparación es simplemente una multiplicación de vectores, y con esto se obtendrá un *puntaje* que mide el *grado de asociación entre pares de palabras*.   

<img src="./img/comparacion-queries-keys-red-transformer.jpeg">   

Así, para el caso de la frase que queremos traducir, si analizamos la palabra “Italian” los *puntajes* obtenidos indican que al codificar este token se le debería prestar más atención a la propia palabra “Italian” seguida por la palabra “food”, y se debería enfocar menos en las palabras “love” y “I”, que tienen los menores *puntajes*.

La idea es ahora usar estos *puntajes* para ponderar cada uno de los vectores `values`, indicando así la importancia de cada palabra al momento de la codificación de los tokens.

Para poder hacer esto se debe escalar los *puntajes*, dividiéndolos primero entre el tamaño de cada vector, y luego pasándolos a una función softmax. Esta función permite simplemente representar cada valor como una probabilidad entre cero y uno:   

<img src="./img/aplicacion-funcion-softmax-red-transformer.jpeg">   


Un valor cercano a uno indica que la red debe prestarle más atención a ese token en particular, y un valor cercano a 0 que la palabra no es muy relevante.

Finalmente, se debe condensar toda esta información resultante de la comparación en un solo vector por cada token.    

Así se coge la matriz de puntajes que se acaba de obtener y se multiplica por la matriz de `values`: el resultado serán cuatro nuevos tokens, que contendrán la codificación de la información de contexto más relevante para cada palabra de la secuencia:   

<img src="./img/salida-bloque-atencional-red-transformer.jpeg">    


En resumen, el bloque de atención toma los tokens iniciales y codifica en los tokens resultantes los elementos de la secuencia a los que se debe dar más relevancia.

Sin embargo, cabe recordar que en la frase original se encuentra, además de asociaciones entre palabras, asociaciones entre frases: para traducir la porción “Italian food” se necesita prestar atención a “I love”.

Así que un solo bloque de atención no es suficiente. Al usar múltiples bloques de atención es posible detectar y codificar asociaciones entre palabras y grupos de palabras a diferentes niveles.

Las salidas de estos bloques se combinan en una última red neuronal que condensa toda la información resultante en un único vector para cada token de entrada:    

<img src="./img/multiples-bloques-atencionales-red-transformer.jpeg">   


Hasta aquí, el bloque de atención.

### Codificación + Bloque residual   

El siguiente bloque que encontramos en esta arquitectura es el *bloque residual*. A este bloque llega tanto la entrada como la salida del bloque de atención, y esto se hace ya que la red es muy profunda y si únicamente se enviara la salida la información, esta se degradaría progresivamente y esto dificultaría el entrenamiento y desempeño de la red.

En esta etapa se cogen los dos datos, se suman y luego se normalizan para que tengan la forma adecuada requerida por el siguiente bloque:   

<img src="./img/bloque-residual-codificador-red-transformer.jpeg">   


### Codificación: Red Neuronal + Bloque Residual   

Este nuevo bloque consta de una Red Neuronal seguida por un bloque residual:   

<img src="./img/red-neuronal-y-bloque-residual-codificador-red-transformer.jpeg">   

La red neuronal procesa en paralelo todos los vectores de la secuencia, tomando la información atencional de las capas anteriores y consolidándola en una única representación.     

La entrada y la salida de esta red neuronal son luego entradas de un bloque residual que tiene exactamente las mismas características del bloque anterior: una suma seguida por una normalización de los datos.

### Codificación y resultado final.   

Este bloque parte de los tokens de entrada, los procesa en paralelo y entrega a la salida una representación que contiene información atencional sobre las diferentes relaciones entre palabras o grupos de palabras de la secuencia, importantes al momento de la traducción:   

<img src="./img/codificador-resultado-final-red-transformer.jpeg">  

Y este proceso se repite para los codificadores restantes, que son idénticos en estructura al codificador que se acaba de analizar.

## Decodificadores (Bloque de Decodificación)   

Este segundo bloque es el responsable de llevar a cabo la traducción:    

<img src="./img/decodificador-red-transformer-diagrama-de-bloques.jpeg">    


En primer lugar están los bloques de embedding de salida y un codificador posicional, que cumplen exactamente la misma función de los bloques que se han visto en la etapa de codificación.

A continuación se encuentra el **decodificador**, que es muy similar al bloque de codificación: en total cuenta con 6 decodificadores, cada uno de ellos conectado al codificador, lo que permite conocer la información atencional de la entrada, en el idioma original, para poder realizar la traducción.

Cada decodificador es similar a los bloques de codificación que se han visto antes: cuenta con bloques de atención, residuales y redes neuronales que tienen la misma estructura de los codificadores. Sin embargo tienen un bloque de atención de enmascaramiento y un bloque residual adicionales.

Luego viene una capa lineal que, junto con la capa softmax, permite generar una a una las palabras de la secuencia de salida.   


Seguidamente, se analiza cómo funciona paso a paso la decodificación.

### Decodificación: bloque de atención con enmascaramiento.   

La traducción comienza con la palabra clave “inicio”, la cual es codificada con el embedding y posicionalmente.

Al pasar al primer decodificador es procesada por el bloque atencional de enmascaramiento. Este bloque es prácticamente idéntico al bloque de atención visto anteriormente: codifica la relación entre diferentes elementos de la secuencia de salida, usando los `queries`, `keys` y `values` vistos anteriormente. Pero con una diferencia importante: como se está generando cada palabra de manera secuencial, una a una, el decodificador debe prestar atención *únicamente a la palabra generada actualmente y a las anteriores, no a las futuras*.   

<img src="./img/bloque-atencional-enmascaramiento-decodificador-red-transformer.jpeg">   


Por ejemplo, si en la secuencia traducida nos ubicamos en la palabra “la”, el decodificador debería tener acceso a esta palabra y a “amo”, pero no a palabras que aparecerán posteriormente en la secuencia (“comida” e “italiana”):   

<img src="./img/bloque-enmascaramiento-palabras-a-enmascarar-decodificador-red-transformer.jpeg">   


Para evitar esto, se agrega un bloque que enmascara, es decir que simplemente hace cero, las palabras a las que durante la decodificación no se debe prestar atención:   

<img src="./img/resultado-enmascaramiento-decodificador-red-transformer.jpeg">   

Al igual que con el codificador, en este caso también se emplean múltiples bloques de atención para detectar relaciones a diferentes niveles.

### Decodificación: bloque de atención.   

Todos los bloques residuales, así como la red neuronal de este decodificador funcionan de forma idéntica a como funcionan en los codificadores.

Toca centrarse ahora en el bloque atencional que, en este caso, tiene la misma estructura pero un funcionamiento ligeramente diferente al del codificador.

Este bloque enfoca su atención tanto en la secuencia original como en la de salida y para ello toma la salida del codificador y las pasa a las redes `“queries”` y `“keys”`, mientras que el nodo `“values”` usa como entrada el dato proveniente del bloque residual anterior:   

<img src="./img/bloque-atencional-decodificador-red-transformer.jpeg">   


Es de esta manera como el codificador le indica al decodificador a qué elementos debe prestar más atención al momento de generar la secuencia de salida.

De nuevo, se usan múltiples bloques de atención de manera simultánea para codificar asociaciones a diferentes niveles.

Hasta aquí el primer decodificador.

### Decodificación: múltiples decodificadores y etapas de salida.   

Este bloque se replica un total de seis veces, y al final genera un vector con cantidades numéricas, y dónde lo único que será necesario es convertirlo en una palabra:   

<img src="./img/multiples-decodificadores-red-transformer.jpeg">   


Para ello se usa, en primer lugar, la capa lineal, que es simplemente una red neuronal que coge el vector producido por el decodificador y lo transforma en un vector mucho más grande.

Por ejemplo, si el traductor aprende 10000 palabras (es decir el tamaño del vocabulario), entonces el vector de salida de la capa lineal tendrá precisamente 10000 elementos.   


La capa softmax coge cada elemento de este vector y lo convierte en una probabilidad, todas con valores positivos entre 0 y 1.    

La posición con la probabilidad más alta será seleccionada y la palabra asociada con dicha posición será precisamente la salida del modelo en ese instante de tiempo:   

<img src="./img/capas-lineal-y-softmax-decodificacion-red-transformer.jpeg">   

Este proceso se repite hasta que se genera la totalidad de la secuencia de salida.

## Conclusión   

El ejemplo utilizado en este análisis se basa en un caso concreto de traducción pero cabe resaltar que este tipo de arquitectura funciona para muchos otros tipos de tareas y problemas más allá de los problemas sobre NLP.